In [1]:
import myFunctions as mf
import pandas as pd
import numpy as np
import json

In [26]:
myData = pd.read_csv("preprocessed_dataFile.csv").drop(['Unnamed: 0', 'encounter_id'],axis=1)[0:20000]
col = myData.columns
print('Number of rows: ', len(myData))
print('Number of columns: ', len(col))

Number of rows:  20000
Number of columns:  21


In [27]:
myDataA = myData[col[0:11]]
myDataB = myData[col[11:22]].drop('diag_3', axis=1)
myDataB['diag_3'] = myData['diag_3']
Y = myDataB['diag_3']

In [28]:
# myDataA = pd.read_csv("https://raw.githubusercontent.com/sunchang0124/PPDML/master/Privacy-preserving%20bayesians/preprocessed_dataFile_A.csv")
# myDataA = myDataA.drop('Unnamed: 0', axis=1)

In [29]:
# We assume target class is at B site and always the last column in B data
# myDataB = pd.read_csv("https://raw.githubusercontent.com/sunchang0124/PPDML/master/Privacy-preserving%20bayesians/preprocessed_dataFile_B.csv")
# colB = myDataB.columns
# Y = myDataB['diag_3']
# myDataB = myDataB[colB[0:6]].drop(['Unnamed: 0'], axis=1) # we only use first 5 columns here because the values    


In [30]:
Divide_set = 10
C_seed = 2
C_min = 0
C_max = 5

In [31]:
% time resultStageOne = mf.start_at_A(myDataA, Divide_set, C_seed, C_min, C_max)

CPU times: user 4min 40s, sys: 7min 57s, total: 12min 37s
Wall time: 17min 49s


In [32]:
% time resultStageTwo = mf.start_at_B(myDataB, C_seed, C_min, C_max, resultStageOne["sumNoiseBytes"], Divide_set)

CPU times: user 17min, sys: 12min 45s, total: 29min 45s
Wall time: 46min 8s


In [33]:
% time resultStageThree = mf.communication_at_A(myDataA, resultStageOne['randomBytes'], resultStageTwo['sumNoisesAB'], \
                                         resultStageTwo['sumNoisesB'], Divide_set)


CPU times: user 241 ms, sys: 45.8 ms, total: 286 ms
Wall time: 407 ms


In [34]:
% time myResult = mf.Final_at_B(myDataB, resultStageThree["randomsSumSet"], resultStageThree["sumNoisesBARand"], \
                         resultStageThree["XaTXa"], resultStageTwo['randomBytes'], Divide_set)


XaTXa: 12 - 12
XbTXa: 12 - 1
XaTXb: 9 - 1
XbTXb: 10 - 1
XbTXb_exclY: 9 - 1
Coefficients: 
 [[  2.91930788e+00]
 [  1.72473308e+00]
 [ -1.16096993e+01]
 [ -1.08742468e-02]
 [  3.42081088e+00]
 [  3.47961492e-01]
 [ -2.63128874e-01]
 [ -5.58430798e-01]
 [ -6.04279254e-01]
 [ -2.18479192e+00]
 [  1.55155637e-01]
 [  7.22721117e-01]
 [ -1.27336665e-02]
 [  2.87124158e-02]
 [ -1.20701878e+00]
 [  2.53808931e+00]
 [  1.73470332e+00]
 [ -9.58211863e+00]
 [  1.70954501e+01]
 [  4.61929588e+01]]
Intercept:  285.22904500129516
CPU times: user 112 ms, sys: 26.4 ms, total: 138 ms
Wall time: 94.9 ms


In [ ]:
##### checking with centralized datasets by using scikit learn #####

X_centralized = np.concatenate((myDataA, myDataB.drop('diag_3', axis=1)), axis=1)
from sklearn import linear_model
regr = linear_model.LinearRegression(fit_intercept=True, normalize=True)
regr.fit(X_centralized, Y)

# The coefficients
print('Coefficients: \n', regr.coef_)
print('Intercept: \n', regr.intercept_)

In [ ]:
# for i in range(0, len(myResult["coefficients"])):
#     print(str(np.matrix(regr.coef_[i+1])) == str(myResult["coefficients"][i]))